<a href="https://colab.research.google.com/github/Nidhinavi/NLP/blob/main/Sentiment_Analysis_Using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pandas textblob nltk scikit-learn torch torchvision torchaudio torchtext datasets tokenizers torchmetrics tensorboard altair wandb spacy torchinfo

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,dataloader
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import nltk
import re
import string
from nltk.corpus import stopwords
from collections import Counter
## Libraries Preposeeing for NLP task
nltk.download('all')
nltk.download('Stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test csv.csv",encoding='latin1')

In [ ]:
df


,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0
...,...,...,...,...,...,...,...,...,...
4810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
##We will be keeping only two features from the dataset
df=df[['text','sentiment']]

In [ ]:
df.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [ ]:
##We will always ensure our target table has a balance data
df['sentiment'].value_counts()

,count
sentiment,
neutral,1430
positive,1103
negative,1001


In [ ]:
## Check the null value
df.isnull().sum()

,0
text,1281
sentiment,1281


In [ ]:
df=df.dropna().reset_index(drop=True)

In [ ]:
df.isnull().sum()

,0
text,0
sentiment,0


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.shape

(3534, 2)

####Preprossing for NLP:
1. Lower the text
2. Remove punctuation
3. Removing stopwords
4. Removing URL

#####We will create a funtion which will fill all the requirements

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk import sent_tokenize


######performing encoding on target table as the need to convert into numeric form like 0,1,2


In [ ]:
sentiment={'negative':0,'neutral':1,'positive':2}

In [ ]:
df['sentiment_encoded'] = df['sentiment'].map(sentiment).fillna(-1).astype(int)

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
def preprocessing_text(text):
    if isinstance(text, str):
        # Lowercasing
        text = text.lower()

        # Removing URLs
        text = re.sub(r'(http|https|ftp|ssh)://[\w_-]+(?:\.[\w_-]+)+[\w.,@?^=%&:/~+#-]*', '', text)

        # Removing punctuations
        text = re.sub(r'[^\w\s]', '', text)

        # Removing stopwords and applying lemmatization
        stop_words = set(stopwords.words('english'))
        text_lst = text.split()
        processed_words = [lemmatizer.lemmatize(word) for word in text_lst if word not in stop_words]

        return ' '.join(processed_words)
    else:
        return ""

# Apply function to DataFrame
df['preprocessed_text'] = df['text'].apply(preprocessing_text)

In [ ]:
# Fix: Ensure sent_tokenize is not overridden
word = []  # Initialize list for tokenized words

for text in df['preprocessed_text']:  # Iterate over processed text
    sentences = sent_tokenize(text)  # Tokenize into sentences (Ensure it's not overwritten!)
    for sentence in sentences:
        word.append(simple_preprocess(sentence))  # Tokenize into words and append

print(word)  # Output the processed words

[['last', 'session', 'day'], ['shanghai', 'also', 'really', 'exciting', 'precisely', 'skyscraper', 'galore', 'good', 'tweeps', 'china', 'sh', 'bj'], ['recession', 'hit', 'veronique', 'branquinho', 'quit', 'company', 'shame'], ['happy', 'bday'], ['like'], ['thats', 'great', 'weee', 'visitor'], ['think', 'everyone', 'hate', 'lol'], ['soooooo', 'wish', 'could', 'im', 'school', 'myspace', 'completely', 'blocked'], ['within', 'short', 'time', 'last', 'clue'], ['get', 'day', 'alright', 'havent', 'done', 'anything', 'yet', 'leaving', 'soon', 'stepsister', 'though'], ['bike', 'put', 'holdshould', 'known', 'argh', 'total', 'bummer'], ['checked', 'didnt', 'win'], ['youre', 'twitter', 'tavern', 'bore', 'much'], ['im', 'va', 'weekend', 'youngest', 'son', 'turn', 'tomorrowit', 'make', 'kinda', 'sad', 'getting', 'big', 'check', 'twipics'], ['coming', 'socket', 'feel', 'like', 'phone', 'hole', 'virgin', 'thats', 'loose'], ['hot', 'today', 'dont', 'like', 'hate', 'new', 'timetable', 'bad', 'week'], ['

In [ ]:
import gensim
###Train the model
model = gensim.models.Word2Vec(word)

In [ ]:
model.wv.index_to_key

['im',
 'day',
 'good',
 'get',
 'like',
 'love',
 'go',
 'got',
 'dont',
 'work',
 'know',
 'going',
 'cant',
 'today',
 'time',
 'one',
 'back',
 'lol',
 'happy',
 'see',
 'really',
 'much',
 'think',
 'want',
 'well',
 'night',
 'miss',
 'oh',
 'new',
 'still',
 'mother',
 'feel',
 'thanks',
 'thats',
 'need',
 'last',
 'home',
 'sad',
 'hope',
 'great',
 'would',
 'haha',
 'thing',
 'ive',
 'make',
 'ill',
 'twitter',
 'tomorrow',
 'morning',
 'right',
 'wish',
 'could',
 'fun',
 'though',
 'friend',
 'gonna',
 'tonight',
 'better',
 'bad',
 'nice',
 'come',
 'getting',
 'sorry',
 'look',
 'mom',
 'never',
 'week',
 'watching',
 'didnt',
 'weekend',
 'sleep',
 'people',
 'hour',
 'even',
 'hey',
 'way',
 'school',
 'bed',
 'youre',
 'yes',
 'made',
 'say',
 'guy',
 'let',
 'little',
 'life',
 'next',
 'best',
 'hate',
 'working',
 'first',
 'wait',
 'sick',
 'tweet',
 'yeah',
 'show',
 'suck',
 'movie',
 'already',
 'year',
 'phone',
 'keep',
 'live',
 'song',
 'thank',
 'ur',
 'ta

In [ ]:
model.corpus_count

3523

In [ ]:
model.epochs

5

In [ ]:
model.wv.similar_by_word('good')

[('dont', 0.9992779493331909),
 ('im', 0.9992684125900269),
 ('like', 0.9992288947105408),
 ('going', 0.999198853969574),
 ('go', 0.9991940855979919),
 ('really', 0.9991247653961182),
 ('get', 0.9991151094436646),
 ('day', 0.9990804195404053),
 ('got', 0.9990713000297546),
 ('need', 0.9990509748458862)]

In [ ]:
model.wv['day']

array([-1.19966358e-01,  2.58436948e-01,  1.00481570e-01,  3.87584977e-02,
        1.10362202e-01, -5.03616631e-01,  3.69319357e-02,  5.72781563e-01,
       -2.06131488e-01, -1.02265052e-01, -2.11152688e-01, -2.86404997e-01,
       -6.15025125e-02,  6.92055002e-02,  4.75762375e-02, -2.46254489e-01,
        4.05674689e-02, -2.58448780e-01, -1.25404760e-01, -6.30455434e-01,
        1.98162705e-01,  1.34051532e-01,  1.63416311e-01, -1.99471444e-01,
       -9.21629220e-02,  4.82108109e-02, -2.30235741e-01, -2.25685939e-01,
       -2.23652273e-01,  1.95456184e-02,  3.71077418e-01,  9.00298730e-02,
        1.24121740e-01, -1.06604561e-01, -1.74995586e-01,  2.86627710e-01,
       -5.76524138e-02, -2.74063081e-01, -1.76226676e-01, -5.24439454e-01,
        4.52183262e-02, -1.63393602e-01, -2.04715624e-01, -1.50980996e-02,
        2.87215233e-01, -9.30751935e-02, -1.90610722e-01, -6.15668744e-02,
        2.10352227e-01,  1.11021750e-01,  1.29868433e-01, -2.31929958e-01,
       -2.50812769e-02, -

In [ ]:
model.wv['good'].shape

(100,)

In [ ]:
word[0]

['last', 'session', 'day']

In [ ]:
word[1]

['shanghai',
 'also',
 'really',
 'exciting',
 'precisely',
 'skyscraper',
 'galore',
 'good',
 'tweeps',
 'china',
 'sh',
 'bj']

In [ ]:
def avg_Wrod2vec(doc):
  valid_vectors=[model.wv[word] for word in doc if word in model.wv]
  if len(valid_vectors)==0:
    return np.zeros(100)
  return np.mean(valid_vectors,axis=0)

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
import numpy as np
X=[]
for sentence in tqdm(word):
  X.append(avg_Wrod2vec(sentence))

100%|██████████| 3523/3523 [00:00<00:00, 37012.67it/s]


In [ ]:
X

[array([-0.10083391,  0.22683388,  0.09229648,  0.03168662,  0.09054607,
        -0.4487248 ,  0.03089911,  0.5065447 , -0.18429986, -0.09120008,
        -0.1863938 , -0.24660073, -0.05423341,  0.06020917,  0.0406553 ,
        -0.22433329,  0.02933376, -0.23396951, -0.10907628, -0.55905557,
         0.17935605,  0.12491345,  0.14473951, -0.17757647, -0.08620143,
         0.03837044, -0.20004013, -0.19982626, -0.2014058 ,  0.01475733,
         0.33054996,  0.0835022 ,  0.1111557 , -0.0967778 , -0.15290937,
         0.2519113 , -0.05747089, -0.251009  , -0.16396174, -0.4750669 ,
         0.03212991, -0.14171189, -0.18158856, -0.01498521,  0.26210514,
        -0.08467846, -0.17297904, -0.05434197,  0.19313711,  0.09901515,
         0.11588489, -0.19869399, -0.02776896, -0.05337353, -0.21411552,
         0.13762295,  0.0990712 , -0.00977317, -0.3014536 ,  0.10457584,
         0.13428362,  0.04970045, -0.03337568, -0.0950213 , -0.3844098 ,
         0.2419347 ,  0.08216146,  0.22447905, -0.3

In [ ]:
len(X)

3523

In [ ]:
X=np.array(X)

In [ ]:
type(X)

numpy.ndarray

In [ ]:
X.shape

(3523, 100)

In [ ]:
df=df[['preprocessed_text','sentiment_encoded']]

In [ ]:
df.shape

(3534, 2)

In [ ]:
df.head()

,preprocessed_text,sentiment_encoded
0,last session day,1
1,shanghai also really exciting precisely skyscr...,2
2,recession hit veronique branquinho quit compan...,0
3,happy bday,2
4,like,2


In [ ]:
df=df[list(map(lambda x:len(x)>0,df['preprocessed_text']))]

In [ ]:
df.shape

(3523, 2)

In [ ]:
df

,preprocessed_text,sentiment_encoded
0,last session day,1
1,shanghai also really exciting precisely skyscr...,2
2,recession hit veronique branquinho quit compan...,0
3,happy bday,2
4,like,2
...,...,...
3529,3 im tired cant sleep try,0
3530,alone old house thanks net keep alive kicking ...,2
3531,know mean little dog sinking depression want m...,0
3532,_sutra next youtube video gonna love video,2


In [ ]:
X

array([[-0.10083391,  0.22683388,  0.09229648, ..., -0.29719737,
         0.06595679, -0.00525205],
       [-0.04559097,  0.1029708 ,  0.03709683, ..., -0.13367435,
         0.0299983 ,  0.00052847],
       [-0.01494118,  0.04119784,  0.01404772, ..., -0.05344052,
         0.01991339, -0.00471484],
       ...,
       [-0.05328358,  0.11672666,  0.05143627, ..., -0.15716223,
         0.03918433, -0.00253234],
       [-0.04926972,  0.11478508,  0.04988542, ..., -0.15657589,
         0.0353834 , -0.00597751],
       [-0.0245777 ,  0.05380154,  0.0214966 , ..., -0.06982587,
         0.0205623 ,  0.00344655]])

In [ ]:
Y=df['sentiment_encoded']

In [ ]:
Y

,sentiment_encoded
0,1
1,2
2,0
3,2
4,2
...,...
3529,0
3530,2
3531,0
3532,2


In [ ]:
Y.shape

(3523,)

In [ ]:
Y=np.array(Y)

In [ ]:
type(Y)

numpy.ndarray

In [ ]:
Y.shape

(3523,)

In [ ]:
Y

array([1, 2, 0, ..., 0, 2, 2])

In [ ]:
X_train,X_test,Y_train, Y_test=train_test_split(X,Y, test_size=0.2,random_state=42)

In [ ]:
X_train.shape

(2818, 100)

In [ ]:
X_test.shape

(705, 100)

In [ ]:
Y_train.shape

(2818,)

In [ ]:
Y_test.shape

(705,)

In [ ]:
import torch

In [ ]:
from torch.utils.data import DataLoader,TensorDataset

In [ ]:
##Convert Numpy array to Pytorch Tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(Y_train, dtype=torch.long)  # Use long for classification labels
y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

####Create PyTorch Dataset and DataLoader (for Sentence Embeddings)
#####The Dataset and DataLoader are simplified because we are now dealing with fixed-size sentence embeddings, not sequences. No padding needed!

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        # embedding is already a tensor
        embedding = self.embeddings[idx]
        label = self.labels.iloc[idx]
        return embedding, torch.tensor(label)

In [ ]:
# Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [ ]:
## Example of batch size
for embedding_batch, label_batch in train_loader:
    print("Embedding batch shape:", embedding_batch.shape)
    print("Label batch shape:", label_batch.shape)
    break

Embedding batch shape: torch.Size([32, 100])
Label batch shape: torch.Size([32])


In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
df['preprocessed_text	'] = list(preprocessed_text)


NameError: name 'preprocessed_text' is not defined

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, embedding_dim, output_dim, hidden_dim, dropout_prob):
        super().__init__()

        # First fully connected layer
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)

        # Second fully connected layer
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, embedding):

        # Input is sentence embedding directly
        # embedding shape: [batch_size, embedding_dim]
        out = self.fc1(embedding)
        out = self.relu(out)
        out = self.dropout(out)

        # No softmax here, CrossEntropyLoss will handle it
        out = self.fc2(out)
        return out

# Model parameters
embedding_dim = X.shape[1]
hidden_dim = 256
output_dim = len(df['sentiment_encoded'])
dropout_prob = 0.5

model = SentimentClassifier(embedding_dim, output_dim, hidden_dim, dropout_prob)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(model)

SentimentClassifier(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=3523, bias=True)
)


In [ ]:
# Hyper parameters
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
num_epochs = 100

def calculate_accuracy(predictions, labels):
    "Calculates the accuracy of predictions."

    # Get the class with highest probability
    _, predicted_classes = torch.max(predictions, 1)
    correct_predictions = (predicted_classes == labels).sum().item()
    accuracy = correct_predictions / len(labels)
    return torch.tensor(accuracy)

def train(model, dataloader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for batch_idx, (embedding_batch, label_batch) in enumerate(dataloader):
        embedding_batch = embedding_batch.to(device)
        label_batch = label_batch.to(device)

        optimizer.zero_grad()
        predictions = model(embedding_batch) # Pass embeddings
        loss = criterion(predictions, label_batch)
        acc = calculate_accuracy(predictions, label_batch)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)


def evaluate(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for batch_idx, (embedding_batch, label_batch) in enumerate(dataloader):
            embedding_batch = embedding_batch.to(device)
            label_batch = label_batch.to(device)

            predictions = model(embedding_batch)
            loss = criterion(predictions, label_batch)
            acc = calculate_accuracy(predictions, label_batch)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%')

Epoch: 01, Train Loss: 4.0604, Train Acc: 37.18%, Test Loss: 1.3940, Test Acc: 41.44%
Epoch: 02, Train Loss: 1.2992, Train Acc: 38.45%, Test Loss: 1.2475, Test Acc: 41.44%
Epoch: 03, Train Loss: 1.2180, Train Acc: 36.24%, Test Loss: 1.1505, Test Acc: 41.44%
Epoch: 04, Train Loss: 1.1563, Train Acc: 38.90%, Test Loss: 1.1186, Test Acc: 41.44%
Epoch: 05, Train Loss: 1.1276, Train Acc: 38.45%, Test Loss: 1.0928, Test Acc: 35.60%
Epoch: 06, Train Loss: 1.1083, Train Acc: 39.85%, Test Loss: 1.0936, Test Acc: 36.96%
Epoch: 07, Train Loss: 1.1023, Train Acc: 39.89%, Test Loss: 1.0866, Test Acc: 37.50%
Epoch: 08, Train Loss: 1.1009, Train Acc: 38.83%, Test Loss: 1.0872, Test Acc: 37.77%
Epoch: 09, Train Loss: 1.0999, Train Acc: 39.15%, Test Loss: 1.0844, Test Acc: 38.32%
Epoch: 10, Train Loss: 1.0931, Train Acc: 39.43%, Test Loss: 1.0918, Test Acc: 41.44%
Epoch: 11, Train Loss: 1.0897, Train Acc: 40.17%, Test Loss: 1.0826, Test Acc: 41.44%
Epoch: 12, Train Loss: 1.0941, Train Acc: 39.01%, Test

In [ ]:
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f'Final Test Loss: {test_loss:.4f}, Test Acc: {test_acc*100:.2f}%')

Final Test Loss: 1.0720, Test Acc: 36.82%
